In [22]:
!pip install yfinance

https://pypi.org/project/yfinance/ 

In [23]:
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt

In [24]:
# Commodities
Wheat_SRW = yf.Ticker('ZW=F').history(period="max")
Soybeans = yf.Ticker('ZS=F').history(period="max")

In [25]:
# Financial
Canadian_Dollar = yf.Ticker('6C=F').history(period="max")
British_Pound = yf.Ticker('6B=F').history(period="max")

In [32]:
Wheat_SRW

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-07-17,248.00,248.00,243.75,244.00,7233,0,0
2000-07-18,244.25,247.50,241.00,241.25,6523,0,0
2000-07-19,241.25,246.00,239.00,245.00,7136,0,0
2000-07-20,244.00,247.50,244.00,247.00,3792,0,0
2000-07-21,247.00,248.50,246.50,247.25,4380,0,0
...,...,...,...,...,...,...,...
2021-03-19,629.00,635.50,621.75,627.00,51669,0,0
2021-03-22,627.50,630.75,621.25,627.25,36124,0,0
2021-03-23,628.00,644.50,621.00,634.75,68769,0,0
